In [6]:
import json
import re
from collections import defaultdict

In [2]:
with open("od_edm_dataset_metadata.json") as f:
    mds = json.load(f)

In [3]:
datasets = {}
for ds in mds:
    datasets[ds['id']] = ds

In [4]:
n_of_words = 4
column_info = []
n = 0
for ds in mds:
    if 'survey' not in ds['tags']:
        n+=1
        for col in ds['columns']:
            cname = col['name'].strip().replace('_',' ')
            if not re.match(r".+[0-9]+",cname):
                if len(cname.split(' '))<=n_of_words:
                    entry = {'id':cname, 'dataset':ds['id'], 'type':col['type'], 'desc':col['description']}
                    column_info.append(entry)
print("Total ds: ",len(mds))
print("Processed ds: ",n)

Total ds:  584
Processed ds:  506


In [5]:
import random

pos_ds = random.sample(mds,200)

In [2]:
import pandas as pd

In [7]:
n = 0
names = ['id','type','desc','category','dataset','name']
mat_ds = []
for ds in pos_ds:
    if 'survey' not in ds['tags']:
        n+=1
        for col in ds['columns']:
            cname = col['name'].strip().replace('_',' ')
            if not re.match(r".+[0-9]+",cname):
                if len(cname.split(' '))<=n_of_words:
                    entry = {'id':cname, 'dataset':ds['id'], 'type':col['type'], 'desc':col['description'],'name':ds['name'],'category':""}
                    row = [entry[nm] for nm in names]
                    mat_ds.append(row)
print("Total ds: ",len(pos_ds))
print("Processed ds: ",n)
ds_df = pd.DataFrame(mat_ds,columns=names)

Total ds:  200
Processed ds:  171


In [9]:
ds_df.to_csv('golden_ground_no_cat.csv',index=False)

In [3]:
golden_ground_df = pd.read_csv("manual_category_concept.csv")
golden_ground = golden_ground_df[pd.notna(golden_ground_df.concept)].copy()

In [4]:
names = golden_ground.columns
ground = []
cat_cluster = {}
concept_cluster = {}
for row in golden_ground.as_matrix():
    rdict = dict(zip(names,row))
    cat = rdict['category']
    cat_cluster.setdefault(cat,[])
    cat_cluster[cat].append(rdict)
    conc = rdict['concept']
    concept_cluster.setdefault(conc,[])
    concept_cluster[conc].append(rdict)
    ground.append(rdict)

In [7]:
with open("ground.json",'w') as f:
    json.dump(ground,f,indent=4)
with open("ground_by_cat.json",'w') as f:
    json.dump(cat_cluster,f,indent=4)
with open("ground_by_concept.json",'w') as f:
    json.dump(concept_cluster,f,indent=4)
    